<a href="https://colab.research.google.com/github/nreichen/eecs448-CollegeSubredditsCommentGeneration/blob/main/subredditValidation/msu_test_val.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

import os 
GOOGLE_DRIVE_PATH_AFTER_MY_DRIVE = "448 Group: Jedi Order/models/msu"
GOOGLE_DRIVE_PATH = os.path.join('drive', 'My Drive', GOOGLE_DRIVE_PATH_AFTER_MY_DRIVE)
print(os.listdir(GOOGLE_DRIVE_PATH))

Mounted at /content/drive
['checkpoints', 'train_post_set.pickle', 'val_post_set.pickle', 'test_post_set.pickle', 'train_com_set.pickle', 'val_com_set.pickle', 'test_com_set.pickle', 'msu_model.ipynb', 'msu_test_val.ipynb']


In [ ]:
print(GOOGLE_DRIVE_PATH)

drive/My Drive/448 Group: Jedi Order/models/msu


In [ ]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append(GOOGLE_DRIVE_PATH)

In [ ]:
print(GOOGLE_DRIVE_PATH)

drive/My Drive/448 Group: Jedi Order/models/msu


In [ ]:
import locale
print(locale.getpreferredencoding())

UTF-8


In [ ]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [ ]:
!pip install transformers
!pip install convokit
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 83.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 106.2 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.0/168.0 kB 10.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 kB 28.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.4/175.4 kB 23.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 7.9 MB/s eta 0:00:00
  Created wheel for convokit: filename=convokit-2.5.3-py3-none-any.whl size=

In [ ]:
import pickle
import pandas as pd
from transformers import GPT2LMHeadModel, GPT2Tokenizer, TrainingArguments, Trainer, DataCollatorForLanguageModeling, EarlyStoppingCallback
from datasets import load_dataset
import nltk
# maybe modify the above to a different model? idk up to you all
# also how to use the gpu command? like the cuda stuff.
# 
from convokit import Corpus, download
from torch.utils.data import DataLoader, IterableDataset, Dataset
import torch
import numpy as np

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


# Load and Prepare Dataset


In [ ]:
seed = 448
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)

In [ ]:
print(GOOGLE_DRIVE_PATH)

drive/My Drive/448 Group: Jedi Order/models/msu


In [ ]:
val_post_path = os.path.join(GOOGLE_DRIVE_PATH, "val_post_set.pickle")
val_com_path = os.path.join(GOOGLE_DRIVE_PATH, "val_com_set.pickle")

In [ ]:
print(val_post_path)

drive/My Drive/448 Group: Jedi Order/models/msu/val_post_set.pickle


In [ ]:
# posts_datasets_to_use[0] = pickle.load(open(train_post_path, "rb"))
val_posts = pickle.load(open(val_post_path, "rb"))
# posts_datasets_to_use[2] = pickle.load(open(test_post_path, "rb"))
# comments_datasets_to_use[0] = pickle.load(open(train_com_path, "rb"))
val_comments = pickle.load(open(val_com_path, "rb"))
# comments_datasets_to_use[2] = pickle.load(open(test_com_path, "rb"))

In [ ]:
for pt in val_posts:
  print(pt)

<bos> I seem to remember Windows 10 being free for MSU students but I can't find the link where I saw it. Does anyone know what I'm talking about and where to find it? <eos> 
<bos> Currently trying to do some research on the best ways to transfer from out of state to in-state the fastest ways, I know it probably won't be too easy to do. Any advice is extremely appreciated EDIT: Is there anyone who i can talk to on campus? <eos> 
<bos> Hi, I am a family of 3 (1 infant), starting work at msu in october. I am looking for a 2br (around 800-1000ft). I am doing my math to estimate how much i will pay on top of rent but i am wondering how much everyone is paying for their electricity and gas. I used to live in upstate new york in a very poorly insulated house so in the winters it usually costs around $300 for heat and electricity.  Could you guys share how much u are paying for heat and electricity in east lansing? Especially in the winter time.  <eos> 
<bos> Who are the best professors for m

In [ ]:
print(val_posts)

["<bos> I seem to remember Windows 10 being free for MSU students but I can't find the link where I saw it. Does anyone know what I'm talking about and where to find it? <eos> ", "<bos> Currently trying to do some research on the best ways to transfer from out of state to in-state the fastest ways, I know it probably won't be too easy to do. Any advice is extremely appreciated EDIT: Is there anyone who i can talk to on campus? <eos> ", '<bos> Hi, I am a family of 3 (1 infant), starting work at msu in october. I am looking for a 2br (around 800-1000ft). I am doing my math to estimate how much i will pay on top of rent but i am wondering how much everyone is paying for their electricity and gas. I used to live in upstate new york in a very poorly insulated house so in the winters it usually costs around $300 for heat and electricity.  Could you guys share how much u are paying for heat and electricity in east lansing? Especially in the winter time.  <eos> ', '<bos> Who are the best profe

Load the trained model

In [ ]:
model_file_path = "checkpoints"

In [ ]:
OUTPUT_DIR = os.path.join(GOOGLE_DRIVE_PATH, model_file_path)
model = GPT2LMHeadModel.from_pretrained(OUTPUT_DIR)

In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2", bos_token = "<bos>", eos_token ="<eos>", truncation_side='right')
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


1

^^^^^^run if needed 

In [ ]:
#https://colab.research.google.com/github/borisdayma/huggingtweets/blob/master/huggingtweets-demo.ipynb#scrollTo=ZSCf6QyF8AG-
# post_in = "I want to go to hotel management school for rural bed and breakfasts. What experiences or classes should I seek out?"
post_in = "Hey everyone! For those of you who took BIOMI 2900, or General Microbiology, did you take the discussion? If so, did it help at all? Trying to decide if I should or not. I'm not a microbiology major but I'm in a microbio lab at the moment so some of it does make sense already. Thanks!"
post_with_bos = post_in + " TL;DR"
encoded_prompt = tokenizer(post_in, return_tensors="pt").input_ids
encoded_prompt = encoded_prompt.to(model.device)

# prediction
output_sequences = model.generate(
    input_ids=encoded_prompt,
    max_new_tokens=75,
    min_new_tokens=10,
    temperature=1.,
    top_p=0.95,
    do_sample=True,
    num_return_sequences=3
)

generated_sequences = []
print(len(output_sequences))
torch.unique(output_sequences, dim=0)
print(len(output_sequences))

# decode prediction
for generated_sequence_idx, generated_sequence in enumerate(output_sequences):
    generated_sequence = generated_sequence.tolist()
    text = tokenizer.decode(generated_sequence, clean_up_tokenization_spaces=True, skip_special_tokens=True)
    #if not ALLOW_NEW_LINES:
        #limit = text.find('\n')
        #text = text[: limit if limit != -1 else None]
    generated_sequences.append(text.strip())

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


3
3


In [ ]:
# print(generated_sequences)
for sent in generated_sequences:
  print(sent.replace(post_in, '').replace('TL;DR', ''))

   The professor at MSU and I had a conversation about how it goes down in class. No need to go into this because there is no more questions, and you can get your answers as quickly as you want from the professor. The best way to learn your way around a micro microbiology lab is to take a few notes from a class that
   You could do something similar but be aware of your options. I'm in another lab and it definitely helps to understand how different classes are. I'll be going out later this week on Monday (I'll start talking about all things related to a microbiology lab here).
   If you had an interesting experience during your freshman year (and I would assume since you were in the engineering program), congrats! We took an easy 5 semester course that was taken with MSU, and it was easy. I highly recommend taking it once every few years. This course is very fun and full of interesting questions, so make sure


In [ ]:
!pip install rouge-score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24954 sha256=e4a484a1ff5cc928b17f2ab84bedddb0b3ec0c427aedc4f327aafc9e93dce9cb
  Stored in directory: /root/.cache/pip/wheels/9b/3d/39/09558097d3119ca0a4d462df68f22c6f3c1b345ac63a09b86e
Successfully built rouge-score


In [ ]:
from rouge_score import rouge_scorer
from torch import optim
from torch.utils.data import Dataset
from tqdm import tqdm

In [ ]:
# print(post_tensors_data[1])
# print(comments_datasets_to_use[1])
def validation_generate(post):
  # print(post)
  # post_with_bos = "<bos>" + post + " TL;DR"
  post_with_bos = post + " TL;DR"
  encoded_prompt = tokenizer(post_with_bos, return_tensors="pt").input_ids
  encoded_prompt = encoded_prompt.to(model.device)
  # print(encoded_prompt)

  # prediction
  output_sequences = model.generate(
    input_ids=encoded_prompt,
    max_new_tokens=75,
    min_new_tokens=10,
    temperature=1.,
    top_p=0.95,
    do_sample=True,
    num_return_sequences=1
  )
    
  generated_sequences = []
  # decode prediction
  for generated_sequence_idx, generated_sequence in enumerate(output_sequences):
    generated_sequence = generated_sequence.tolist()
    text = tokenizer.decode(generated_sequence, clean_up_tokenization_spaces=True, skip_special_tokens=True)
    generated_sequences.append(text.strip())
  
  # print(generated_sequences)
  return generated_sequences

In [ ]:
val_len = len(val_comments)
val_dataset = [{} for sub in range(val_len)]

for cnt, item in enumerate(val_comments):
  val_dataset[cnt]['reference_summary'] = item.replace("<bos>", "").replace("<eos>", "")


print(val_dataset[1]['reference_summary'])

 You have to have proof of residence in Michigan for an entire year without going to school anywhere. That is the only way they begin to consider it. I grew up in Indiana but was born in Michigan and have half my family living up here including my biological dad. So, I had to apply for in-state residence by basically proving that I partially lived in Michigan and have ties here.  


In [ ]:
for cnt, post in enumerate(val_posts):
  # print(post)
  generated_sequences = validation_generate(post)

  for sent in generated_sequences:
    sent = sent.replace(post, '').replace('TL;DR', '')
    val_dataset[cnt]['generated_text'] = sent
    # print(sent)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generati

**Calculating the Rouge score**

In [ ]:
!pip3 install Rouge

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
generated_text = val_dataset[7]['generated_text']
reference_summary = val_dataset[7]['reference_summary']
print("GENERATED COMMENTS: {}\nReference COMMENTS:{}".format(generated_text, reference_summary))

GENERATED COMMENTS: I'm trying to find an apartment close to campus for 3 or 4 people, advice?    For some people you may have to walk or walk to the grocery store.
Reference COMMENTS: [url link] Try dtnmgt.com and use their property search. Some people seem to hate DTN, but honestly I haven't had a problem with them (this will be my 3rd year in a DTN property). A good majority of the close-to-campus properties will be DTN owned. I'm living in a 4 bedroom apartment off of Cedar Street right now, but I'm almost positive all the units in the building are rented out for the next year.  


In [ ]:
from rouge import Rouge

# create a Rouge object
rouge = Rouge()

# initialize scores for each variant of ROUGE
scores_rouge1 = {'recall': 0, 'precision': 0, 'fmeasure': 0}
scores_rouge2 = {'recall': 0, 'precision': 0, 'fmeasure': 0}
scores_rougeL = {'recall': 0, 'precision': 0, 'fmeasure': 0}

for idx in range(val_len):
    generated_text = val_dataset[idx]['generated_text']
    reference_summary = val_dataset[idx]['reference_summary']
    
    scores = rouge.get_scores(generated_text, reference_summary, avg=True)
    
    # compute ROUGE-1 scores
    scores_rouge1['recall'] += scores['rouge-1']['r']
    scores_rouge1['precision'] += scores['rouge-1']['p']
    scores_rouge1['fmeasure'] += scores['rouge-1']['f']
    
    # compute ROUGE-2 scores
    # scores = rouge.get_scores(generated_text, reference_summary, avg=True, rouge_n=(2, 2))
    scores_rouge2['recall'] += scores['rouge-2']['r']
    scores_rouge2['precision'] += scores['rouge-2']['p']
    scores_rouge2['fmeasure'] += scores['rouge-2']['f']
    
    # compute ROUGE-L scores
    # scores = rouge.get_scores(generated_text, reference_summary, avg=True, rouge_l=True)
    scores_rougeL['recall'] += scores['rouge-l']['r']
    scores_rougeL['precision'] += scores['rouge-l']['p']
    scores_rougeL['fmeasure'] += scores['rouge-l']['f']

# compute the average scores over all examples
n_examples = val_len
scores_rouge1 = {k: v/n_examples for k, v in scores_rouge1.items()}
scores_rouge2 = {k: v/n_examples for k, v in scores_rouge2.items()}
scores_rougeL = {k: v/n_examples for k, v in scores_rougeL.items()}

# print the overall scores
print("Overall ROUGE-1 scores:", scores_rouge1)
print("Overall ROUGE-2 scores:", scores_rouge2)
print("Overall ROUGE-L scores:", scores_rougeL)

Overall ROUGE-1 scores: {'recall': 0.2408403406295535, 'precision': 0.1950422368447068, 'fmeasure': 0.20016467239832486}
Overall ROUGE-2 scores: {'recall': 0.028953868690475143, 'precision': 0.022712058587013362, 'fmeasure': 0.02286010129601485}
Overall ROUGE-L scores: {'recall': 0.21164325089640199, 'precision': 0.17141267667791787, 'fmeasure': 0.17544655982790433}


**Calculating the sentiment analysis accuracy**

In [ ]:
!pip3 install vaderSentiment

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 9.1 MB/s eta 0:00:00


In [ ]:
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


In [ ]:
def find_label(scores):
  if scores['pos'] > scores['neg'] and scores['pos'] > scores['neu']:
    dominant_label = 'positive'
  elif scores['neg'] > scores['pos'] and scores['neg'] > scores['neu']:
    dominant_label = 'negative'
  else:
    dominant_label = 'neutral'
  
  return dominant_label

In [ ]:
acc = 0
sid = SentimentIntensityAnalyzer()
for idx in range(val_len):
  # print(val_dataset[idx]['reference_summary'])
  # print(type(val_dataset[idx]['reference_summary']))
  sent_gt = sid.polarity_scores(val_dataset[idx]['reference_summary'])
  sent_generated_comment = sid.polarity_scores(val_dataset[idx]['generated_text'])

  gt_label = find_label(sent_gt)
  generated_comment_label = find_label(sent_generated_comment)

  if gt_label == generated_comment_label:
    acc += 1

  # print("Reference comments: {}".format(val_dataset[idx]['reference_summary']))
  # print(sent_gt)
  # print(gt_label)
  # print("Generated comments: {}".format(val_dataset[idx]['generated_text']))
  # print(sent_generated_comment)
  # print(generated_comment_label)
  # if idx == 15:
  #   break

print("The number of generated comments that have aligned sentiments: {}".format(acc))
print("The number of posts in the validation set: {}".format(val_len))
print("The accuracy of sentiments of the generated comments: {}".format(acc / val_len))

The number of generated comments that have aligned sentiments: 751
The number of posts in the validation set: 765
The accuracy of sentiments of the generated comments: 0.9816993464052287


**Readability validation test**

Automated readability index

In [ ]:
!pip3 install textstat

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.1/105.1 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 68.9 MB/s eta 0:00:00


Flesch-Kincaid Grade Level: The Flesch-Kincaid Grade Level measures the approximate grade level of a text, based on its sentence length and average number of syllables per word.

In [ ]:
import textstat

In [ ]:
grade_level = 0.0
for idx in range(val_len):
  sc = textstat.flesch_kincaid_grade(val_dataset[idx]['generated_text'])
  # print(sc)
  grade_level += sc

grade_level /= val_len
print("Flesch-Kincaid Grade Level:", grade_level)

Flesch-Kincaid Grade Level: 5.715032679738561


Gunning Fog Index: The Gunning Fog Index measures the approximate years of education needed to understand a text, based on its sentence length and percentage of complex words.

In [ ]:
fog_index = 0.0
for idx in range(val_len):
  sc = textstat.gunning_fog(val_dataset[idx]['generated_text'])
  # print(sc)
  fog_index += sc

fog_index /= val_len
print("Gunning Fog Index:", fog_index)

Gunning Fog Index: 7.81001307189542


Automated Readability Index (ARI): The Automated Readability Index measures the approximate grade level of a text, based on its average number of characters per word and average number of words per sentence.



In [ ]:
ari = 0.0
for idx in range(val_len):
  sc = textstat.automated_readability_index(val_dataset[idx]['generated_text'])
  # print(sc)
  ari += sc

ari /= val_len
print("Automated Readability Index:", ari)

Automated Readability Index: 6.161960784313727
